In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
store_info=pd.read_csv('/content/drive/MyDrive/data storm 4.0/Store-info.csv')
testing_data=pd.read_csv('/content/drive/MyDrive/data storm 4.0/Testing-data.csv')
transaction=pd.read_csv('/content/drive/MyDrive/data storm 4.0/Historical-transaction-data.csv')

In [ ]:
transaction=transaction.drop_duplicates()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

known_data = transaction.dropna(subset=['item_description'])
unknown_data = transaction[transaction['item_description'].isna()].copy()

known_datas = known_data.groupby(['item_description'])['item_price'].mean().reset_index()

X_train = known_datas['item_price'].values.reshape(-1, 1)
y_train = known_datas['item_description'].values
clf = RandomForestClassifier().fit(X_train, y_train)

X_test = unknown_data['item_price'].values.reshape(-1, 1)
predicted_name = clf.predict(X_test)

unknown_data['item_description'] = predicted_name

transaction = pd.concat([known_data, unknown_data], axis=0)

In [ ]:
transaction.dropna(subset=['invoice_id'], inplace=True)

In [ ]:
threshold = 1.5

for item in transaction['item_description'].unique():
    subset = transaction[transaction['item_description'] == item]
    # subset_mean = subset['item_price'].mean()
    q1 = subset['item_price'].quantile(0.25)
    q3 = subset['item_price'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - threshold * iqr
    upper_bound = q3 + threshold * iqr
    outliers = subset[(subset['item_price'] < lower_bound) | (subset['item_price'] > upper_bound)]
    rep = subset['item_price'].median()
    outliers.loc[outliers['item_price'] < lower_bound, 'item_price'] = rep
    outliers.loc[outliers['item_price'] > upper_bound, 'item_price'] = rep
    transaction.update(outliers)

In [ ]:
for shop in transaction['shop_id'].unique():
  trans = transaction[transaction['shop_id'] == shop]
  for item in trans['item_description'].unique():
      subset = trans[trans['item_description'] == item]
      subset_mean = subset['quantity_sold'].mean()
      q1 = subset['quantity_sold'].quantile(0.25)
      q3 = subset['quantity_sold'].quantile(0.75)
      iqr = q3 - q1
      lower_bound = subset_mean - threshold * iqr
      upper_bound = subset_mean + threshold * iqr
      if(lower_bound < 0):
        lower_bound = 0
      outliers = subset[(subset['quantity_sold'] < lower_bound) | (subset['quantity_sold'] > upper_bound)]
      rep = subset['quantity_sold'].median()
      outliers.loc[outliers['quantity_sold'] < lower_bound, 'quantity_sold'] = rep
      outliers.loc[outliers['quantity_sold'] > upper_bound, 'quantity_sold'] = rep
      transaction.loc[outliers.index, 'quantity_sold'] = outliers['quantity_sold']


In [ ]:
customers_per_shop = transaction.groupby('shop_id')['customer_id'].nunique().reset_index()

In [ ]:
items_per_shop = transaction.groupby('shop_id')['item_description'].nunique().reset_index()

In [ ]:
transaction['total_price'] = transaction['item_price'] * transaction['quantity_sold']
rev_for_shop = transaction.groupby('shop_id')['total_price'].sum().reset_index()

In [ ]:
transaction7 = pd.get_dummies(transaction, columns=['item_description'])
transaction8 = transaction7.groupby(["shop_id"]).sum().reset_index()
transactions9 = transaction8.drop(columns=['invoice_id','item_price','quantity_sold','total_price'])
transactions9

<ipython-input-33-00875c932e3a>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  transaction8 = transaction7.groupby(["shop_id"]).sum().reset_index()


,shop_id,item_description_BOTTLED DRINKING WATER 1.5L,item_description_BOTTLED DRINKING WATER 500ML,item_description_CHOCOLATE MILK 180ML,item_description_CREAM SODA 1.5L,item_description_CREAM SODA 1L,item_description_CREAM SODA 500ML,item_description_CREAM SODA APPLE POP 1L,item_description_DRY GINGER ALE PET 500ML,item_description_FIT O MANGO 1L,...,item_description_ORANGE BARLEY 1.5L,item_description_ORANGE CRUSH 1.5L,item_description_ORANGE CRUSH 1L,item_description_ORANGE CRUSH 500ML,item_description_SODA 1L,item_description_SODA 500ML,item_description_SODA PET 1.5L,item_description_STRAWBERRY MILK 180ML,item_description_TONIC PET 500ML,item_description_TWISTEE APPLE 1L
0,SHOP001,28,133,0,194,16,48,66,11,24,...,126,32,10,16,58,71,38,15,54,13
1,SHOP002,66,290,0,503,41,236,116,46,41,...,232,104,13,45,129,190,207,24,150,28
2,SHOP003,73,241,0,325,48,156,155,23,62,...,169,66,7,29,107,140,121,36,134,21
3,SHOP004,44,164,0,279,33,105,71,24,31,...,164,62,11,34,127,155,103,38,81,33
4,SHOP005,62,273,0,337,33,143,140,35,65,...,155,76,8,35,126,112,122,49,84,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,SHOP123,41,141,0,173,18,76,76,16,39,...,97,37,8,19,63,60,52,42,42,22
120,SHOP124,87,269,0,254,34,132,101,74,92,...,189,69,12,31,128,179,169,103,139,49
121,SHOP125,41,188,0,41,11,83,27,31,19,...,47,9,14,26,91,124,95,3,93,13
122,SHOP126,74,173,0,189,0,142,106,97,52,...,170,36,0,28,168,224,118,56,157,31


In [ ]:
df_final=pd.merge(store_info,transactions9, on=['shop_id'], how='outer')
df_final=pd.merge(df_final,rev_for_shop, on=['shop_id'], how='outer')
df_final=pd.merge(df_final,items_per_shop, on=['shop_id'], how='outer')
df_final=pd.merge(df_final,customers_per_shop, on=['shop_id'], how='outer')

In [ ]:
df_final

,shop_id,shop_area_sq_ft,shop_profile,item_description_BOTTLED DRINKING WATER 1.5L,item_description_BOTTLED DRINKING WATER 500ML,item_description_CHOCOLATE MILK 180ML,item_description_CREAM SODA 1.5L,item_description_CREAM SODA 1L,item_description_CREAM SODA 500ML,item_description_CREAM SODA APPLE POP 1L,...,item_description_ORANGE CRUSH 500ML,item_description_SODA 1L,item_description_SODA 500ML,item_description_SODA PET 1.5L,item_description_STRAWBERRY MILK 180ML,item_description_TONIC PET 500ML,item_description_TWISTEE APPLE 1L,total_price,item_description,customer_id
0,SHOP047,528,Moderate,22,114,0,144,17,69,62,...,20,47,32,59,16,63,15,502430.0,36,923
1,SHOP009,676,High,63,440,0,394,39,147,171,...,67,84,129,79,21,74,34,1275015.0,36,2467
2,SHOP083,676,Low,44,234,0,338,38,121,137,...,31,87,83,80,41,64,36,1012325.0,36,1885
3,SHOP117,676,Low,55,193,0,270,31,78,202,...,27,243,233,245,44,176,23,1282015.0,36,2021
4,SHOP042,676,Low,51,235,0,286,24,117,119,...,30,89,124,88,44,53,31,878020.0,36,1825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,SHOP087,527,NaN,19,119,0,201,9,80,65,...,18,44,60,72,15,64,17,575190.0,35,1073
120,SHOP050,411,NaN,39,144,0,501,0,135,143,...,37,111,123,141,22,82,21,1297730.0,32,1766
121,SHOP061,699,NaN,46,110,0,312,20,66,126,...,15,101,94,148,46,67,37,953345.0,36,1727
122,SHOP056,597,NaN,82,336,0,604,53,219,228,...,58,79,147,93,94,84,39,1719525.0,36,2784


In [ ]:
predict = df_final[df_final['shop_profile'].isnull()]
train = df_final[df_final['shop_profile'].notnull()]

In [ ]:
train_new=train.drop('shop_id', axis=1)

In [ ]:
predict_new = predict.drop('shop_id', axis=1)
predict_new = predict_new.drop('shop_profile', axis=1)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier()

df = train_new
X_train = df.drop('shop_profile', axis=1)
y_train = df['shop_profile']
X_test= predict_new

gb_clf.fit(X_train, y_train)
y_pred = gb_clf.predict(X_test)



# Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(df.drop('shop_profile', axis=1), df['shop_profile'], test_size=0.2, random_state=42)

# Train a logistic regression classifier on the training set

# Make predictions on the test set
y_pred = gb_clf.predict(X_test)
y_pred

array(['Low', 'High', 'Low', 'Moderate', 'Moderate', 'Low', 'Moderate',
       'Moderate', 'Moderate', 'Moderate', 'High', 'Low', 'Low', 'High',
       'Moderate', 'Low', 'Low', 'Low', 'High', 'Moderate', 'Low', 'Low',
       'High', 'Low'], dtype=object)

In [ ]:
result_df = pd.DataFrame({'shop_id': predict['shop_id'], 'shop_profile': y_pred})
result_df

,shop_id,shop_profile
100,SHOP046,Low
101,SHOP024,High
102,SHOP023,Low
103,SHOP097,Moderate
104,SHOP044,Moderate
105,SHOP030,Low
106,SHOP038,Moderate
107,SHOP029,Moderate
108,SHOP096,Moderate
109,SHOP092,Moderate


In [ ]:
result_df.to_csv('result.csv', index=False)